# 06: Procesamiento de Diputados y Biografías

**Propósito:** Este *notebook* toma todos los archivos `diputados_bio.csv` (crudos, por período) de `data/01_raw/`, y los transforma en un único archivo maestro de diputados limpio.

**Proceso:**
1.  **Carga y Consolidación:** Lee todos los archivos `diputados_bio.csv` y los une.
2.  **Limpieza y Estandarización:** Normaliza los campos extraídos por el LLM (ej. `universidad`, `maximo_nivel_educativo`) usando mapeos y *fuzzy matching*.
3.  **Deduplicación:** Crea un registro único por `Diputado.Id`, seleccionando la biografía de mayor calidad (mejor `match_score`).
4.  **Guardado:** Guarda el archivo maestro en `data/02_processed/`.

**Dependencias:**
* `data/01_raw/[periodo]/diputados_bio.csv` (Múltiples archivos)

**Salidas (Artifacts):**
* `data/02_processed/diputados_master_clean.parquet` (Un único archivo)

In [1]:
import pandas as pd
from pathlib import Path
import sys
import logging
from tqdm.notebook import tqdm # Para progress_apply

# --- Configurar Logging ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Importar lógica personalizada de /src ---
sys.path.append('../') 
try:
    from src.processing_utils import (
        load_all_bio_files, 
        standardize_education,
        deduplicate_deputies,
        standardize_civil_status,
        standardize_location,
        extract_dependencia_colegio
    )
except ImportError as e:
    logging.error(f"ERROR: No se pudieron importar las funciones desde /src. {e}")
    raise

# Registrar 'tqdm' con pandas
tqdm.pandas()

In [14]:
# --- 1. Configuración de Rutas y Constantes ---
ROOT = Path.cwd().parent
DATA_DIR_RAW = ROOT / "data" / "01_raw"
DATA_DIR_PROCESSED = ROOT / "data" / "02_processed"

# Asegurarse que el directorio de salida exista
DATA_DIR_PROCESSED.mkdir(parents=True, exist_ok=True)

MASTER_COLEGIOS = DATA_DIR_RAW / "colegios chile.csv"
# Archivo de salida
OUTPUT_FILE = DATA_DIR_PROCESSED / "diputados_master_clean.parquet"

logging.info(f"Directorio Raw: {DATA_DIR_RAW}")
logging.info(f"Directorio Processed: {DATA_DIR_PROCESSED}")
logging.info(f"Archivo de Salida: {OUTPUT_FILE}")

2025-10-27 15:46:23,821 - INFO - Directorio Raw: C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-\data\01_raw
2025-10-27 15:46:23,823 - INFO - Directorio Processed: C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-\data\02_processed
2025-10-27 15:46:23,824 - INFO - Archivo de Salida: C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-\data\02_processed\diputados_master_clean.parquet


## 1. Carga y Consolidación

Cargamos todos los archivos `diputados_bio.csv` de todos los períodos en un solo DataFrame.

In [ ]:
try:
    df_colegios_chile = pd.read_csv(MASTER_COLEGIOS)
    logging.info(f"Se cargó la lista maestra de {len(df_colegios_chile)} períodos.")
    display(df_periodos.head())
except FileNotFoundError as e:
    logging.error(f"ERROR FATAL: No se encontró el archivo de dependencia: {MASTER_COLEGIOS}")
    raise

In [3]:
# Llamar a la función del módulo /src
df_full = load_all_bio_files(DATA_DIR_RAW)

if not df_full.empty:
    display(df_full.head())
    print(f"Dimensiones del DataFrame consolidado: {df_full.shape}")
else:
    logging.error("No se cargaron datos. Deteniendo el notebook.")
    # raise Exception("No se cargaron datos.")

2025-10-27 14:38:32,845 - INFO - Buscando archivos 'diputados_bio.csv'...
2025-10-27 14:38:32,847 - INFO - Encontrados 7 archivos. Cargando...
2025-10-27 14:38:32,927 - INFO - DataFrame consolidado creado con 929 filas.


,FechaInicio,FechaTermino,Diputado.Id,Diputado.Nombre,Diputado.Nombre2,Diputado.ApellidoPaterno,Diputado.ApellidoMaterno,Diputado.FechaNacimiento,Diputado.FechaDefucion,Diputado.RUT,...,madre,estado_civil,numero_total_hijos,colegios,universidad,carrera,maximo_nivel_educativo,trabajo,fuente_periodo,Distrito
0,2002-03-10,NaN,1,Mario,NaN,Acuña,Cisternas,NaN,NaN,NaN,...,María Cisterna Fuentealba,Casado/a,3.0,[],Universidad Austral de Chile,Ingeniero Agrónomo,Educación Universitaria,[],1998-2002,NaN
1,2002-03-10,NaN,3,Gustavo,NaN,Alessandri,Valdés,NaN,NaN,NaN,...,Verónica Balmaceda,Casado/a,3.0,['Colegio de los Sagrados Corazones de Manqueh...,Universidad de California,Maquinaria agrícola,Educación Universitaria,"['Obrero', 'Empresario independiente', 'Propie...",1998-2002,NaN
2,2002-03-10,NaN,8,Rafael,NaN,Arratia,Valdebenito,NaN,NaN,NaN,...,Olivia Valdebenito Cuevas,Casado/a,4.0,['Colegio Manuel León Prado'],Universidad de Chile,Médico cirujano,Educación Universitaria,"['Médico oftalmólogo', 'Director Médico de la ...",1998-2002,NaN
3,2002-03-10,NaN,10,Nelson,NaN,Avila,Contreras,NaN,NaN,NaN,...,María Olivia Contreras Chinchón,Casado,2.0,"['Colegio Las Carmelitas', 'Liceo de Hombres d...",Universidad de Chile,Administrador Público,Educación Universitaria,['Jefe de personal en Entel Chile'],1998-2002,NaN
4,2002-03-10,NaN,11,Francisco,NaN,Bartolucci,Johnston,NaN,NaN,NaN,...,Josefina Jhonston Miranda,Casado/a,5.0,['Colegio San Pedro Nolasco'],Universidad Católica de Valparaíso,Ciencias Jurídicas y Sociales,Educación Universitaria,['Profesor auxiliar de Derecho Romano'],1998-2002,NaN


Dimensiones del DataFrame consolidado: (929, 43)


In [4]:
to_drop = ['FechaTermino', 'Diputado.Nombre2', 'Diputado.FechaDefucion', 'Diputado.RUT',
              'Diputado.RUTDV', 'Distrito', 'bio_texto_completo', 'estudios_vida_laboral_parrafos',
              'familia_juventud_parrafos', 'status', 'match_score', 'match_nombre_bcn', 'distrito', 'url_wiki']
df_full.drop(columns=to_drop, inplace=True)

In [5]:
df_full.isna().sum()

FechaInicio                          0
Diputado.Id                          0
Diputado.Nombre                      0
Diputado.ApellidoPaterno             0
Diputado.ApellidoMaterno             0
Diputado.FechaNacimiento            44
Diputado.Sexo._value_1               0
Diputado.Sexo.Valor                  0
Diputado.Militancias.Militancia      0
Distrito.Numero                    320
Distrito.Comunas.Comuna            320
FechaInicio.1                        0
FechaTermino.1                       0
Partido.Id                           0
Partido.Nombre                       0
Partido.Alias                        0
nombre_completo                      0
lugar_nacimiento                    51
fecha_nacimiento                    19
padre                              104
madre                              100
estado_civil                       150
numero_total_hijos                 160
colegios                            10
universidad                         92
carrera                  

In [13]:
df_full['colegios'].value_counts()

colegios
[]                                                                                                            41
['Colegio Tabancura']                                                                                         13
['Colegio Alemán de Santiago']                                                                                10
['Liceo José Victorino Lastarria']                                                                            10
['Colegio Verbo Divino']                                                                                       9
                                                                                                              ..
['Escuela Básica F-93 de Freirina', 'Liceo San Francisco de Vallenar', 'Liceo Antonio Varas de Cauquenes']     1
['Liceo N°7 de Niñas de Providencia, Santiago']                                                                1
['Colegio de la Monjas Francesas']                                                     

## 2. Limpieza y Estandarización

Aplicamos las funciones de limpieza de `src/processing_utils.py` para normalizar los campos extraídos por el LLM.

In [ ]:
if not df_full.empty:
    # --- Limpieza de Tipos (Ejemplo) ---
    to_drop = ['FechaTermino', 'Diputado.Nombre2', 'Diputado.FechaDefucion', 'Diputado.RUT',
              'Diputado.RUTDV', 'Distrito', 'bio_texto_completo', 'estudios_vida_laboral_parrafos',
              'familia_juventud_parrafos', 'status', 'match_score', 'match_nombre_bcn', 'distrito']
    logging.info("Limpiando tipos de datos...")
    df_full['fecha_nacimiento'] = pd.to_datetime(df_full['fecha_nacimiento'], errors='coerce')
    df_full['numero_total_hijos'] = pd.to_numeric(df_full['numero_total_hijos'], errors='coerce').astype('Int64')
    
    df_processed = standardize_education(df_full)
    df_processed['fecha_nacimiento_llm'] = pd.to_datetime(df_processed['fecha_nacimiento'], errors='coerce')
    df_processed['fecha_nacimiento_api'] = pd.to_datetime(df_processed['Diputado.FechaNacimiento'], errors='coerce')
    df_processed['fecha_nac_clean'] = df_processed['fecha_nacimiento_api'].fillna(
            df_processed['fecha_nacimiento_llm']
        )
    df_processed['colegio_egreso_raw'] = df_processed['colegios'].apply(extract_last_colegio)
    df_processed['dependencia'] = df_processed['colegio_egreso_raw'].progress_apply(lambda x: 
                extract_dependencia_colegio(x, df_colegios_chile)
               )
    df_processed['estado_civil_clean'] = df_processed['estado_civil'].progress_apply(lambda x: standardize_civil_status(pd.Series(x)))
    df_location_features = standardize_location(df_processed['lugar_nacimiento'])
    df_processed = df_processed.join(df_location_features)

    logging.info("Procesamiento de campos finalizado.")
    display(df_processed[['universidad', 'universidad_clean', 'maximo_nivel_educativo', 'educacion_nivel_clean']].sample(10))
    df_master = df_processed.drop(columns=to_drop).reset_index(drop=True)
else:
    logging.warning("DataFrame vacío, saltando limpieza.")